In [ ]:
from google.colab import files
uploaded = files.upload()
print('Uploaded files:', uploaded.keys())

# Pick uploaded dataset
DATA_PATH = list(uploaded.keys())[0]
print(f'Using dataset: {DATA_PATH}')

Saving hyphenated hindi.txt to hyphenated hindi.txt
Uploaded files: dict_keys(['hyphenated hindi.txt'])
Using dataset: hyphenated hindi.txt


In [ ]:
# =============================================
# 🧹 CLEAN + INSTALL FIXED DEPENDENCIES (COLAB)
# Works for: mBERT / IndicBERT / XLM-R / BERT
# =============================================

!pip uninstall -y simpletransformers transformers tokenizers sentencepiece seqeval torch torchvision torchaudio fastai peft accelerate jax jaxlib flax optax dopamine-rl orbax-checkpoint chex pytensor umap-learn || true
!pip cache purge -q

# ---- Reinstall clean compatible stack ----
!pip install -q --upgrade pip setuptools wheel
!pip install -q torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121


# Fix for SentencePiece wheel build issue
!apt-get -q install -y libsentencepiece-dev
!pip install -q sentencepiece==0.1.99

# Core NLP libraries
!pip install -q simpletransformers==0.64.3 transformers==4.37.2 tokenizers==0.15.2 seqeval==1.2.2

# ---- Verify installation ----
import torch, transformers, sentencepiece, simpletransformers
print("✅ All dependencies installed successfully!")
print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: fastai 2.8.5
Uninstalling fastai-2.8.5:
  Successfully uninstalled fastai-2.8.5
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-

In [ ]:
import os
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    raw_text = f.read()
lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
print('Example lines:', lines[:5])
print('Total lines:', len(lines))

Example lines: ['"रामने- सीताको- बचाया",', '"भारतका- इतिहास- बहुत- समृद्ध है",', '"मुझे- किताबें पढ़ना- पसंद है",', '"वह- रोज़ सुबह- योग- करता है",', '"बच्चे- पार्क में- खेलते हैं",']
Total lines: 3236


In [ ]:
import re, unicodedata

DETACH_PUNCT = True
END_PUNCT = '।.!?…'

def strip_invisibles(s):
    return ''.join(ch for ch in s if unicodedata.category(ch) not in {'Cf'}).replace('\ufeff', '')

def normalize_hyphens(s):
    # Convert em-dash/en-dash to normal hyphen
    s = s.replace('–', '-').replace('—', '-')
    # Put spaces around hyphens
    s = re.sub(r'\s*-\s*', ' - ', s)
    # Normalize extra spaces
    return re.sub(r'\s+', ' ', s).strip()

def attach_hyphens(s):
    toks, out = s.split(), []
    for t in toks:
        if t == '-' and out:
            out[-1] += '-'   # attach hyphen to previous word
        else:
            out.append(t)
    return ' '.join(out)

def detach_end_punct(s):
    if not DETACH_PUNCT:
        return s
    toks, out = s.split(), []
    for t in toks:
        if len(t) > 1 and t[-1] in END_PUNCT:
            out.append(t[:-1])  # word
            out.append(t[-1])   # punctuation as separate token
        else:
            out.append(t)
    return ' '.join(out)

# ---------------------------------------------------------
# APPLY PREPROCESSING TO 'lines'  (Your previous variable)
# ---------------------------------------------------------

seen = set()
sentences = []

for ln in lines:
    ln = strip_invisibles(ln)
    ln = normalize_hyphens(ln)
    ln = attach_hyphens(ln)
    ln = detach_end_punct(ln)

    if ln not in seen:
        seen.add(ln)
        sentences.append(ln)

print("Total sentences after preprocess:", len(sentences))
print("Example sentences:", sentences[:5])


Total sentences after preprocess: 2240
Example sentences: ['"रामने- सीताको- बचाया",', '"भारतका- इतिहास- बहुत- समृद्ध है",', '"मुझे- किताबें पढ़ना- पसंद है",', '"वह- रोज़ सुबह- योग- करता है",', '"बच्चे- पार्क में- खेलते हैं",']


In [ ]:
import re, unicodedata
DETACH_PUNCT = True
END_PUNCT = '।.!?…'

def strip_invisibles(s):
    return ''.join(ch for ch in s if unicodedata.category(ch) not in {'Cf'}).replace('\ufeff','')

def normalize_hyphens(s):
    s = s.replace('–', '-').replace('—', '-')
    s = re.sub(r'\s*-\s*', ' - ', s)
    return re.sub(r'\s+', ' ', s).strip()

def attach_hyphens(s):
    toks, out = s.split(), []
    for t in toks:
        if t == '-' and out: out[-1] += '-'
        else: out.append(t)
    return ' '.join(out)

def detach_end_punct(s):
    if not DETACH_PUNCT: return s
    toks, out = s.split(), []
    for t in toks:
        if len(t)>1 and t[-1] in END_PUNCT:
            out += [t[:-1], t[-1]]
        else: out.append(t)
    return ' '.join(out)

seen, sentences = set(), []
for ln in lines:
    ln = detach_end_punct(attach_hyphens(normalize_hyphens(strip_invisibles(ln))))
    if ln not in seen:
        seen.add(ln)
        sentences.append(ln)
print('Total sentences after preprocess:', len(sentences))

Total sentences after preprocess: 1845


In [ ]:
# ==================================================
# 🧩 Fix NumPy-Pandas Binary Incompatibility (Colab)
# ==================================================

!pip uninstall -y numpy pandas scikit-learn || true
!pip install -U pip setuptools wheel
!pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.5.1 --force-reinstall

# Verify after reinstall
import numpy, pandas, sklearn
print("✅ NumPy:", numpy.__version__)
print("✅ Pandas:", pandas.__version__)
print("✅ Scikit-learn:", sklearn.__version__)


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 146.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 109.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 54.2 MB/s  0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: threadpoolctl
    Found ex

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_s, test_s = train_test_split(sentences, test_size=0.20, random_state=42)
train_s, val_s = train_test_split(train_s, test_size=0.10, random_state=42)

def make_ner_df(sent_list, labeled=True):
    rows=[]
    for sid, s in enumerate(sent_list):
        for tok in s.split():
            rows.append({
                'sentence_id': sid,
                'words': tok[:-1] if tok.endswith('-') else tok,
                'labels': 'HYPHEN' if tok.endswith('-') else 'NO-HYPHEN'
            } if labeled else {'sentence_id': sid, 'words': tok.replace('-', '')})
    return pd.DataFrame(rows)

train_df, val_df = make_ner_df(train_s), make_ner_df(val_s)
test_df_lbl, test_df_unlbl = make_ner_df(test_s), make_ner_df(test_s, labeled=False)
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)
test_df_lbl.to_csv('test_data_labelled.csv', index=False)
test_df_unlbl.to_csv('test_data.csv', index=False)
print('Data prepared and saved!')

Data prepared and saved!


In [ ]:
import torch, gc
from simpletransformers.ner import NERModel, NERArgs
from collections import defaultdict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

LABELS = ['HYPHEN', 'NO-HYPHEN']
USE_CUDA = torch.cuda.is_available()

def low_mem_args(seed=42):
    a = NERArgs()
    a.num_train_epochs = 3
    a.train_batch_size = 2
    a.eval_batch_size = 2
    a.max_seq_length = 32
    a.learning_rate = 4e-5
    a.labels_list = LABELS
    a.evaluate_during_training = True
    a.early_stopping_patience = 2
    a.early_stopping_metric = 'f1_score'
    a.early_stopping_metric_minimize = False
    a.overwrite_output_dir = True
    a.fp16 = True
    a.manual_seed = seed
    return a

def cleanup(model=None, *objs):
    if model: del model
    for o in objs:
        try: del o
        except: pass
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

def train_and_eval(model_type, model_name, out_dir):
    args = low_mem_args()
    args.output_dir = out_dir
    model = NERModel(model_type, model_name, args=args.__dict__, use_cuda=USE_CUDA)
    model.train_model(train_df, eval_data=val_df)
    return model

# ===============================================
# 🚀 Train All Models: mBERT, IndicBERT, BERT, XLM-R
# ===============================================

print('Training models...')

# mBERT — multilingual BERT
mbert = train_and_eval('bert', 'bert-base-multilingual-cased', 'mbert_be_output')

# IndicBERT — use open Google MuRIL (supports Indian languages)
indic = train_and_eval('bert', 'google/muril-base-cased', 'indicbert_be_output')

# BERT (English-cased)
bertc = train_and_eval('bert', 'bert-base-cased', 'bert_cased_be_output')

# XLM-Roberta
xlmr = train_and_eval('xlmroberta', 'xlm-roberta-base', 'xlmr_be_output')



Training models...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Running Epoch 2 of 3:   0%|          | 0/806 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
# ==============================================================
# ✅ Model Evaluation + Save Best Model (Corrected & Colab Ready)
# ==============================================================

import os
import shutil
import numpy as np
import pandas as pd
import editdistance
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


# --------------------------------------------------------------
# Helper functions
# --------------------------------------------------------------
def boundary_edit_distance(true_tokens, pred_tokens):
    tb = {i for i, t in enumerate(true_tokens) if t.endswith('-')}
    pb = {i for i, t in enumerate(pred_tokens) if t.endswith('-')}
    edits = len(tb.symmetric_difference(pb))
    denom = max(len(tb) + len(pb), 1)
    return edits / denom


def average_bed(true_sents, pred_sents):
    return sum(boundary_edit_distance(t.split(), p.split()) for t, p in zip(true_sents, pred_sents)) / len(true_sents)


def reconstruct(df_with_pred, pred_col):
    sents = defaultdict(list)
    for _, row in df_with_pred.iterrows():
        sid, tok, lab = row['sentence_id'], row['words'], row[pred_col]
        sents[sid].append(tok + '-' if lab == 'HYPHEN' else tok)
    return [' '.join(sents[k]) for k in sents.keys()]


def compute_metrics_from_labels(name, gold_df, pred_labels_flat, true_sentences=None, pred_sents=None):
    gold = gold_df['labels'].tolist()

    # --- Fix length mismatch ---
    if len(pred_labels_flat) != len(gold):
        diff = abs(len(pred_labels_flat) - len(gold))
        print(f"⚠️ {name}: Adjusting mismatch (gold={len(gold)}, preds={len(pred_labels_flat)}) by trimming {diff} tokens.")
        if len(pred_labels_flat) > len(gold):
            pred_labels_flat = pred_labels_flat[:len(gold)]
        else:
            gold = gold[:len(pred_labels_flat)]
    # ----------------------------

    # Compute HYPHEN detection metrics
    precision = precision_score(gold, pred_labels_flat, pos_label='HYPHEN', zero_division=0)
    recall = recall_score(gold, pred_labels_flat, pos_label='HYPHEN', zero_division=0)
    f1 = f1_score(gold, pred_labels_flat, pos_label='HYPHEN', zero_division=0)
    acc = accuracy_score(gold, pred_labels_flat)

    # Compute Boundary Edit Distance
    BED = 1.0
    if true_sentences and pred_sents:
        BED = np.mean([
            editdistance.eval(t, p) / max(len(t), len(p))
            for t, p in zip(true_sentences, pred_sents)
        ])

    # Compute sentence-level exact match
    sentence_exact = np.mean([1 if t == p else 0 for t, p in zip(true_sentences, pred_sents)])

    return {'Model': name, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Accuracy': acc, 'BED': BED, 'Sentence_Exact': sentence_exact}


# --------------------------------------------------------------
# Load Test Data
# --------------------------------------------------------------
test_unlbl = pd.read_csv('test_data.csv')
test_lbl = pd.read_csv('test_data_labelled.csv')


# --------------------------------------------------------------
# Evaluate Each Model
# --------------------------------------------------------------
def eval_model(model, model_name):
    # Group test tokens by sentence
    test_sentences = test_unlbl.groupby('sentence_id')['words'].apply(list).tolist()

    # Run predictions
    preds, _ = model.predict(test_sentences, split_on_space=False)
    flat_preds = [list(d.values())[0] for sent in preds for d in sent]

    # Handle length mismatch
    df_eval = test_unlbl.copy()
    if len(flat_preds) < len(df_eval):
        print(f"⚠️ Warning: Trimming {len(df_eval) - len(flat_preds)} extra tokens from test data for {model_name}")
        df_eval = df_eval.iloc[:len(flat_preds)]
    elif len(flat_preds) > len(df_eval):
        print(f"⚠️ Warning: Trimming {len(flat_preds) - len(df_eval)} extra predictions for {model_name}")
        flat_preds = flat_preds[:len(df_eval)]

    df_eval['pred'] = flat_preds

    # Reconstruct sentences
    pred_sents = reconstruct(df_eval, 'pred')
    true_sents = reconstruct(test_lbl.rename(columns={'labels': 'pred'}), 'pred')

    # Compute metrics
    metrics = compute_metrics_from_labels(model_name, test_lbl, flat_preds, true_sents, pred_sents)
    print(f"[{model_name}] BED={metrics['BED']:.4f} | F1={metrics['F1']:.4f}")
    return metrics


# --------------------------------------------------------------
# Run Evaluation Across All Models
# --------------------------------------------------------------
results = [
    eval_model(mbert, 'mBERT'),
    eval_model(indic, 'IndicBERT'),
    eval_model(bertc, 'BERT'),
    eval_model(xlmr, 'XLM-R')
]

results_df = pd.DataFrame(results).sort_values('BED', ascending=True).reset_index(drop=True)

print('\n=== 🧾 Model Results (sorted by lowest BED) ===')
print(results_df[['Model','Precision','Recall','F1', 'BED','Sentence_Exact']].round(4))

# --------------------------------------------------------------
# Save Best Model Automatically
# --------------------------------------------------------------
best_row = results_df.iloc[0]
best_model_name = best_row['Model']
lowest_bed = best_row['BED']
print(f"\n🏆 Best Model: {best_model_name} (BED={lowest_bed:.4f})")

model_dirs = {
    'mBERT': 'mbert_be_output',
    'IndicBERT': 'indicbert_be_output',
    'BERT': 'bert_cased_be_output',
    'XLM-R': 'xlmr_be_output'
}

best_dir = model_dirs[best_model_name]
save_path = 'best_model'

if os.path.exists(save_path):
    shutil.rmtree(save_path)
shutil.copytree(best_dir, save_path)
print(f"✅ Saved best model ({best_model_name}) to '{save_path}/'")


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/224 [00:00<?, ?it/s]

[mBERT] BED=0.0221 | F1=0.8363


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/224 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/224 [00:00<?, ?it/s]

⚠️ Warning: Trimming 22 extra tokens from test data for BERT
⚠️ BERT: Adjusting mismatch (gold=2934, preds=2912) by trimming 22 tokens.
[BERT] BED=0.0797 | F1=0.4208


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/224 [00:00<?, ?it/s]

[XLM-R] BED=0.0220 | F1=0.8341

=== 🧾 Model Results (sorted by lowest BED) ===
       Model  Precision  Recall      F1     BED  Sentence_Exact
0      XLM-R     0.8707  0.8005  0.8341  0.0220          0.4464
1      mBERT     0.8635  0.8107  0.8363  0.0221          0.4643
2  IndicBERT     0.8315  0.8166  0.8240  0.0242          0.3996
3       BERT     0.4334  0.4089  0.4208  0.0797          0.0562

🏆 Best Model: XLM-R (BED=0.0220)
✅ Saved best model (XLM-R) to 'best_model/'


In [ ]:
!zip -r /content/best_model.zip /content/best_model/

  adding: content/best_model/ (stored 0%)
  adding: content/best_model/training_progress_scores.csv (deflated 38%)
  adding: content/best_model/model_args.json (deflated 61%)
  adding: content/best_model/tokenizer_config.json (deflated 75%)
  adding: content/best_model/checkpoint-1328-epoch-2/ (stored 0%)
  adding: content/best_model/checkpoint-1328-epoch-2/scheduler.pt (deflated 55%)
  adding: content/best_model/checkpoint-1328-epoch-2/model_args.json (deflated 61%)
  adding: content/best_model/checkpoint-1328-epoch-2/tokenizer_config.json (deflated 75%)
  adding: content/best_model/checkpoint-1328-epoch-2/vocab.txt (deflated 45%)
  adding: content/best_model/checkpoint-1328-epoch-2/optimizer.pt (deflated 56%)
  adding: content/best_model/checkpoint-1328-epoch-2/special_tokens_map.json (deflated 42%)
  adding: content/best_model/checkpoint-1328-epoch-2/config.json (deflated 53%)
  adding: content/best_model/checkpoint-1328-epoch-2/training_args.bin (deflated 51%)
  adding: content/bes

In [ ]:
from google.colab import files
files.download('/content/best_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh /content/


total 6.6G
drwxr-xr-x 6 root root 4.0K Nov  1 17:17  bert_cased_te_output
drwxr-xr-x 6 root root 4.0K Nov  1 17:09  best_model
-rw-r--r-- 1 root root 6.6G Nov  1 18:39  best_model.zip
drwxr-xr-x 2 root root 4.0K Nov  1 17:23  cache_dir
-rw-r--r-- 1 root root 315K Nov  1 15:57 'hyphenated sentences telugu (1) (1).txt'
-rw-r--r-- 1 root root 315K Nov  1 15:51 'hyphenated sentences telugu (1).txt'
drwxr-xr-x 6 root root 4.0K Nov  1 17:09  indicbert_te_output
drwxr-xr-x 6 root root 4.0K Nov  1 16:16  mbert_te_output
drwxr-xr-x 3 root root 4.0K Nov  1 16:03  outputs
drwxr-xr-x 7 root root 4.0K Nov  1 17:18  runs
drwxr-xr-x 1 root root 4.0K Oct 30 13:36  sample_data
-rw-r--r-- 1 root root  51K Nov  1 16:00  test_data.csv
-rw-r--r-- 1 root root  71K Nov  1 16:00  test_data_labelled.csv
-rw-r--r-- 1 root root 255K Nov  1 16:00  train_data.csv
-rw-r--r-- 1 root root  28K Nov  1 16:00  val_data.csv
drwxr-xr-x 6 root root 4.0K Nov  1 17:40  xlmr_te_output


In [ ]:
import shutil
import os
from google.colab import files

if os.path.exists("/content/best_model.zip"):
    files.download("/content/best_model.zip")
else:
    print("⚠️ File not found! Make sure best_model.zip exists in /content/")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>